### все импорты

In [1]:
import os, json
import pandas as pd

In [2]:
from sklearn import preprocessing  # там OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np

In [4]:
from tqdm import tqdm

In [5]:
from sklearn import ensemble
from sklearn import metrics
import pickle
from datetime import datetime
import time

In [6]:
from sklearn.neighbors import KNeighborsClassifier

### составление таблицы признаков

In [7]:
with open(os.path.join("d:/", "corp.json"), encoding='utf8') as f:
    data = json.load(f)

In [8]:
data['1001']

[[{'link_name': 'punct',
   'feat_p': 'fPOS=PUNCT++,',
   'parent': 1,
   'form': '--',
   'postag_p': 'PUNCT'},
  {'parent': 2,
   'form': 'Не',
   'link_name': 'neg',
   'postag_p': 'PART',
   'lemma': 'не',
   'feat_p': 'fPOS=PART++',
   'sem': '',
   'feat': 'PART',
   'sem2': ''},
  {'parent': -1,
   'form': 'волнуйтесь',
   'link_name': 'ROOT',
   'postag_p': 'VERB',
   'lemma': 'волноваться',
   'feat_p': 'Aspect=Imp|Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act|fPOS=VERB++',
   'sem': 'ca:noncaus t:move',
   'feat': 'V ipf intr med pl 2p imper',
   'sem2': 'ca:noncaus t:psych:emot'},
  {'link_name': 'punct',
   'feat_p': 'fPOS=PUNCT++,',
   'parent': 2,
   'form': ',',
   'postag_p': 'PUNCT'},
  {'parent': 5,
   'form': 'со',
   'link_name': 'case',
   'postag_p': 'ADP',
   'lemma': 'со',
   'feat_p': 'fPOS=ADP++',
   'sem': '',
   'feat': 'PR',
   'sem2': ''},
  {'parent': 8,
   'form': 'мной',
   'link_name': 'nmod',
   'postag_p': 'PRON',
   'lemma': 'я',
   'feat_p':

### для заданного слова достает морфологичекие, синтаксические и семантические фичи

In [9]:
# использовать в цикле с enumerate
def get_features(word):  # слово dict и его индекс в предложении word_ind
    features = {}
    
    # morphology features
    if "feat" not in word:
        return
    

    if "feat" in word: # redundant if
        morph_inf = word["feat"]
        all_m_f = morph_inf.split()  # for 'S n inan sg nom' ['S', 'n', ...]
        pos = ['S', 'A', 
               'NUM', 'ANUM', 
               'SPRO', 'APRO']  
        # сущ прил числ числительное-прилагательное (один, седьмой, восьмидесятый)
        #  местоимение-существительное (она, что)
        # APRO — местоимение-прилагательное (который, твой)
        
        if all_m_f[0] not in pos:
            return
        else:
            features['pos'] = all_m_f[0]
        
        
        # Animacy=Anim|Case=Gen|Gender=Masc|Number=Plur|fPOS=NOUN++',
        case = ['acc', 'loc', 'acc2', 'loc2', 'ins']  # вин(+2вин) твор предл(+2предл) 
        
        if not all_m_f[-1] in case:
            return
        else:
            features['case'] = all_m_f[-1]
        
        features['gender'] = None
        features['animacy'] = None
        features['number'] = None
                
        try: # APRO pl loc
            gender, animacy, number = all_m_f[1], all_m_f[2], all_m_f[3]
            features['gender'] = gender
            features['animacy'] = animacy
            features['number'] = number
        except:
            for i in range(1, len(all_m_f)-1):
                if all_m_f[i] in ['pl', 'sg']:
                    number = all_m_f[i]
                elif all_m_f[i] in ['m', 'n', 'f']:
                    gender = all_m_f[i]
                elif all_m_f[i] in ['inan', 'anim']:
                    animacy = all_m_f[i]
                else:
                    print(all_m_f, word, 'error in gram features')
    
    
    features['lemma'] = word['lemma'] # mb delete this feature bc correlation
    
    
    # syntax features
#     features['word_ind'] = word_ind  # не совсем синтаксис наверн
    
    features['link_name'] = word['link_name']
#     features['parent_ind'] = word['parent'] # индекс слова в предложении
        
    # semantic features
    
    if "sem" in word:
        sem_inf = word['sem']
        all_s_f = sem_inf.split()
        
        # t:hum r:propn t:famn'
        ts = []
        rs = []
        for part in all_s_f:
            if part.startswith('t:'):
                ts.append(part[2:])
            elif part.startswith('r:'):
                rs.append(part[2:])
        
        features['t'] = ' '.join(ts)
        features['r'] = ' '.join(rs)
        
        if not features['t']:
            features['t'] = None
        if not features['r']:
            features['r'] = None
    
    return features


### изначальные таблицы

In [8]:
adj_df = pd.read_csv("framebank_anno_ex_circ.txt", sep='\t')
adj_df.head(10)

,ConstrIndex,ExIndex,PlaceC,Phrase,Form,Role,Type,KeyLexemes
0,11467,32603,1,самым,APRO,степень,Circum,"адъективные_конструкции,близкий,верный,достове..."
1,11467,35305,1,такой,APRO,степень,Circum,"адъективные_конструкции,близкий,верный,достове..."
2,11538,36561,1,не,PART,отрицание,Modal,алкать
3,11538,36563,1,Потом,ADV,последовательность,Circum,алкать
4,11538,36565,1,беспрестанно,ADV,узуальность,Circum,алкать
5,11538,40979,1,так,ADV,степень,Circum,алкать
6,11538,40996,1,страстно,ADV,степень,Circum,алкать
7,11538,41003,1,В божьем деле,в + Sloc,сфера,Circum,алкать
8,11538,41014,1,ныне,ADV,время,Circum,алкать
9,11538,41039,1,уже,ADV,NaN,Modal,алкать


In [37]:
compl_df = pd.read_csv("framebank_anno_ex_items_fixed.txt", sep='\t')
compl_df.head(10)


D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ConstrIndex,ExIndex,Place,PhraseGen,WordDep,Form,Role,Rank,Sem,Rea,ConstrExId,ItemExIndex,KeyLexemes
0,11467,35300,1,None,скверные,A,атрибут,Атрибут,None,None,1452,5163,"адъективные_конструкции,близкий,верный,достове..."
1,11467,35300,2,зубы,зубы,S,носитель признака,Контролер согласования,None,стандартный,1452,5164,"адъективные_конструкции,близкий,верный,достове..."
2,11467,35301,1,None,скверный,A,атрибут,Атрибут,None,None,1453,5173,"адъективные_конструкции,близкий,верный,достове..."
3,11467,35301,2,дурак,дурак,Snom,носитель признака,Контролер согласования,None,стандартный,1453,5174,"адъективные_конструкции,близкий,верный,достове..."
4,11467,32603,1,None,доступным,A,атрибут,Атрибут,None,None,22112,58830,"адъективные_конструкции,близкий,верный,достове..."
5,11467,32603,2,ценам,ценам,Sdat,носитель признака,Контролер согласования,None,стандартный,22112,58831,"адъективные_конструкции,близкий,верный,достове..."
6,11538,36561,1,Ермила с Кирилой,Ермила,Snom,субъект психологического состояния,Субъект,лицо,стандартный,2372,17770,алкать
7,11538,36561,2,None,алкали,алкать,None,Предикат,None,None,2372,17771,алкать
8,11538,36561,3,сказочных чудес,чудес,Sgen,содержание мысли,Объект,None,стандартный,2372,17772,алкать
9,11538,36563,1,пассажиры,пассажиры,Snom,субъект психологического состояния,Несобственный,лицо,контроль,2374,11007,алкать


### удалить то где нет сем роли

In [38]:
compl_df = compl_df[compl_df['Role'] != 'None'].dropna(subset=['Role', 'Rank']) # 40614  # 74816

In [41]:
compl_df = compl_df[compl_df['Rank'] != 'None']

In [52]:
compl_df = compl_df[compl_df['Form'] != 'None'].dropna(subset=['Form'])

In [105]:
adj_df = adj_df[adj_df['Role'] != 'None'].dropna(subset=['Role']) # 40614  # 74816

In [106]:
adj_df = adj_df[adj_df['Form'] != 'None'].dropna(subset=['Form'])

In [109]:
adj_df = adj_df[adj_df['Form'] != 'ADV']

In [112]:
adj_df = adj_df[adj_df['Type'] == 'Circum']

In [104]:
adj_df.columns

Index(['ConstrIndex', 'ExIndex', 'PlaceC', 'Phrase', 'Form', 'Role', 'Type',
       'KeyLexemes'],
      dtype='object')

In [126]:
adj_df.to_csv('adj_df.tsv', index=False, sep='\t')  
compl_df.to_csv('compl_df.tsv', index=False, sep='\t')  

In [128]:
adj_df = pd.read_csv("adj_df.tsv", sep='\t')


In [129]:
compl_df = pd.read_csv("compl_df.tsv", sep='\t')

In [130]:
adj_df.columns

Index(['ConstrIndex', 'ExIndex', 'PlaceC', 'Phrase', 'Form', 'Role', 'Type',
       'KeyLexemes'],
      dtype='object')

In [131]:
compl_df.columns

Index(['ConstrIndex', 'ExIndex', 'Place', 'PhraseGen', 'WordDep', 'Form',
       'Role', 'Rank', 'Sem', 'Rea', 'ConstrExId', 'ItemExIndex',
       'KeyLexemes'],
      dtype='object')

In [132]:
adj_df.head()

,ConstrIndex,ExIndex,PlaceC,Phrase,Form,Role,Type,KeyLexemes
0,11467,32603,1,самым,APRO,степень,Circum,"адъективные_конструкции,близкий,верный,достове..."
1,11467,35305,1,такой,APRO,степень,Circum,"адъективные_конструкции,близкий,верный,достове..."
2,11538,41003,1,В божьем деле,в + Sloc,сфера,Circum,алкать
3,1001,42231,1,дважды,Sacc,частота,Circum,аплодировать
4,1001,42269,1,в нужных местах,в + Sloc,фаза,Circum,аплодировать


In [133]:
compl_df.head()

,ConstrIndex,ExIndex,Place,PhraseGen,WordDep,Form,Role,Rank,Sem,Rea,ConstrExId,ItemExIndex,KeyLexemes
0,11467,35300,1,None,скверные,A,атрибут,Атрибут,None,None,1452,5163,"адъективные_конструкции,близкий,верный,достове..."
1,11467,35300,2,зубы,зубы,S,носитель признака,Контролер согласования,None,стандартный,1452,5164,"адъективные_конструкции,близкий,верный,достове..."
2,11467,35301,1,None,скверный,A,атрибут,Атрибут,None,None,1453,5173,"адъективные_конструкции,близкий,верный,достове..."
3,11467,32603,1,None,доступным,A,атрибут,Атрибут,None,None,22112,58830,"адъективные_конструкции,близкий,верный,достове..."
4,11538,40978,1,рекламистов,рекламистов,Sacc,субъект психологического состояния,Контролер согласования,лицо,активное причастие,3286,11161,алкать


In [147]:
compl_df.head()

,ConstrIndex,ExIndex,PlaceC,PhraseGen,Phrase,Form,Role,Rank,Sem,Rea,ConstrExId,ItemExIndex,KeyLexemes
0,11467,35300,1,None,скверные,A,атрибут,Атрибут,None,None,1452,5163,"адъективные_конструкции,близкий,верный,достове..."
1,11467,35300,2,зубы,зубы,S,носитель признака,Контролер согласования,None,стандартный,1452,5164,"адъективные_конструкции,близкий,верный,достове..."
2,11467,35301,1,None,скверный,A,атрибут,Атрибут,None,None,1453,5173,"адъективные_конструкции,близкий,верный,достове..."
3,11467,32603,1,None,доступным,A,атрибут,Атрибут,None,None,22112,58830,"адъективные_конструкции,близкий,верный,достове..."
4,11538,40978,1,рекламистов,рекламистов,Sacc,субъект психологического состояния,Контролер согласования,лицо,активное причастие,3286,11161,алкать


In [134]:
compl_df.rename({'WordDep': 'Phrase', 'Place':'PlaceC'}, axis=1, inplace=True)

In [135]:
adj_df.rename({'Type': 'Rank'}, axis=1, inplace=True)

In [136]:
adj_df['Rank'].unique()  # Circumstantial Adjunct

array(['Circum'], dtype=object)

In [137]:
adj_df['Rank'] = 'Периферия'

### соединяю в один большой df

In [138]:
compl_df = compl_df[['ExIndex', 'PlaceC', 'Phrase', 'Form', 'Role', 'Rank', 'KeyLexemes']]
adj_df = adj_df[['ExIndex', 'PlaceC', 'Phrase', 'Form', 'Role', 'Rank', 'KeyLexemes']]

In [139]:
# compl_df.info
big_df = compl_df.append(adj_df)

In [140]:
for el in '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~':
    big_df['Phrase'] = big_df['Phrase'].str.replace(el, '', regex=False)

big_df['Phrase'] = big_df['Phrase'].str.replace(' +', ' ', regex=True).str.strip().str.lower()
big_df['Role'] = big_df['Role'].str.replace(' +', ' ', regex=True).str.strip().str.lower()
big_df['Role'] = big_df['Role'].str.replace('метаф.', '', regex=False).str.strip().str.lower()
big_df['Rank'] = big_df['Rank'].str.replace(' +', ' ', regex=True).str.strip().str.lower()

In [141]:
big_df.head()

,ExIndex,PlaceC,Phrase,Form,Role,Rank,KeyLexemes
0,35300,1,скверные,A,атрибут,атрибут,"адъективные_конструкции,близкий,верный,достове..."
1,35300,2,зубы,S,носитель признака,контролер согласования,"адъективные_конструкции,близкий,верный,достове..."
2,35301,1,скверный,A,атрибут,атрибут,"адъективные_конструкции,близкий,верный,достове..."
3,32603,1,доступным,A,атрибут,атрибут,"адъективные_конструкции,близкий,верный,достове..."
4,40978,1,рекламистов,Sacc,субъект психологического состояния,контролер согласования,алкать


In [144]:
big_df = big_df[~big_df['Phrase'].isnull()]

In [145]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27067 entries, 0 to 3313
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ExIndex     27067 non-null  int64 
 1   PlaceC      27067 non-null  int64 
 2   Phrase      27067 non-null  object
 3   Form        27067 non-null  object
 4   Role        27067 non-null  object
 5   Rank        27067 non-null  object
 6   KeyLexemes  27067 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.7+ MB


In [146]:
big_df['Last_word'] = big_df['Phrase'].str.split().str[-1].str.strip()

In [149]:
big_df['Role'] = big_df['Role'].str.split('-').str[0].str.strip()
big_df['Role'] = big_df['Role'].str.split('—').str[0].str.strip()
# '—'


### если роль написана через /, строка разделяется на две

In [150]:
def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [151]:
big_df = splitDataFrameList(big_df,'Role','/')

In [152]:
big_df['Role'] = big_df['Role'].str.strip()

In [156]:
big_df.head()

,ExIndex,PlaceC,Phrase,Form,Role,Rank,KeyLexemes,Last_word
0,35300,1,скверные,A,атрибут,атрибут,"адъективные_конструкции,близкий,верный,достове...",скверные
1,35300,2,зубы,S,носитель признака,контролер согласования,"адъективные_конструкции,близкий,верный,достове...",зубы
2,35301,1,скверный,A,атрибут,атрибут,"адъективные_конструкции,близкий,верный,достове...",скверный
3,32603,1,доступным,A,атрибут,атрибут,"адъективные_конструкции,близкий,верный,достове...",доступным
4,40978,1,рекламистов,Sacc,субъект психологического состояния,контролер согласования,алкать,рекламистов


In [158]:
big_df.drop_duplicates(subset=['ExIndex', 'Phrase', 'Form', 'Role', 'Rank'], inplace=True)

### check the word

In [16]:
def check_word(token, features): # token in all_w_with_role
    
    all_rows = big_df[big_df['Last_word'] == token].to_dict('records') # list with dicts
    
    for row in all_rows:
        row.update(features)
    
    return all_rows

### перебирает корпус, смотрит есть ли слово в адъюнктах/актантах

### new

In [167]:
all_w_with_role = set(big_df['Last_word'])

In [163]:
all_sent_indexes = set(big_df['ExIndex'])

In [164]:
len(all_sent_indexes)

22238

In [173]:
data[1001] # 2 lists inside list

KeyError: 1001

In [168]:
def open_sent(*s):

    for_df = []
    for sent in s: # datav['1001'] - 2 sent 
        
        for i in range(len(sent)): # word == dict
            word = sent[i]
            token = word["form"].lower()
            
            if token in all_w_with_role:
                res = get_features(word)
                if res: 
                    temp = check_word(token, res)  # list of dicts
                    left_cntxt = []
                    right_cntxt = []
                    
#                     if i <= 3:
#                         start = 0
#                     else:
#                         start = i - 3
                    
                    for ind in range(i):
                        left_cntxt.append(sent[ind]["form"].lower())
                    if i < len(sent) - 1:
                        for ind in range(i+1, len(sent)):
                            right_cntxt.append(sent[ind]["form"].lower())
                    
                    left = ' '.join(left_cntxt)
                    right = ' '.join(right_cntxt)
                    
                    for d in temp:
                        d['left_context'] = left
                        d['right_context'] = right
                    
                    for_df += temp
     
    return for_df

In [172]:
all_sent_indexes

[8,
 131107,
 131108,
 131109,
 131110,
 131112,
 131113,
 131114,
 131115,
 131117,
 131118,
 131119,
 131121,
 131122,
 131123,
 131124,
 131125,
 131126,
 131127,
 131128,
 131130,
 131131,
 131132,
 131133,
 131134,
 131136,
 131137,
 131138,
 131139,
 131140,
 131142,
 131144,
 131145,
 131146,
 131147,
 131148,
 131149,
 131150,
 131151,
 131152,
 131153,
 131154,
 131157,
 131158,
 131159,
 131160,
 131161,
 131162,
 131163,
 131165,
 131166,
 131167,
 131168,
 131169,
 131170,
 131171,
 131173,
 131174,
 131175,
 131176,
 131177,
 131178,
 131179,
 131180,
 131181,
 131182,
 131183,
 131184,
 131185,
 131186,
 131187,
 131189,
 131190,
 131191,
 131192,
 131193,
 131194,
 131195,
 131196,
 131197,
 131198,
 131199,
 131200,
 131276,
 131277,
 131278,
 131279,
 131280,
 131281,
 131282,
 131283,
 131284,
 131285,
 131288,
 131291,
 131292,
 131293,
 131294,
 131295,
 131661,
 131664,
 131665,
 131670,
 131673,
 131677,
 131682,
 131755,
 131757,
 131759,
 131760,
 131765,
 13176

collecting features from corpus

In [176]:
final_res = []
all_sent_indexes = list(all_sent_indexes)

for i in tqdm(range(len(all_sent_indexes))):
#     print(type(all_sent_indexes[i]))
    try:
        final_res += open_sent(*data[str(all_sent_indexes[i])])
    except:
        print(str(all_sent_indexes[i]), 'error')
        continue
    # почему - то в 1 раз не записались названия столбцов
    if i == 0:
        final_res_df = pd.DataFrame(final_res)
        final_res_df.to_csv('f_with_context1.tsv', mode='a', index=False, sep='\t') 
        final_res = []
        
    elif i % 500 == 0:
        final_res_df = pd.DataFrame(final_res)
        final_res_df.to_csv('f_with_context1.tsv', mode='a', header=False, index=False, sep='\t') 
        final_res = []
#         print(i)

if final_res:
    final_res_df = pd.DataFrame(final_res)
    final_res_df.to_csv('f_with_context1.tsv', mode='a', header=False, index=False, sep='\t') 
    final_res = []

# final_res_df = pd.DataFrame(final_res)
# final_res_df.to_csv('final_features0.tsv', index=False, sep='\t') 

 12%|█████████▏                                                                   | 2640/22238 [00:39<05:09, 63.31it/s]

30309 error


 28%|█████████████████████▉                                                       | 6337/22238 [03:12<04:33, 58.09it/s]

40958 error


 32%|████████████████████████▉                                                    | 7206/22238 [03:25<02:36, 95.82it/s]

...

 92%|██████████████████████████████████████████████████████████████████████▏     | 20541/22238 [22:56<04:29,  6.30it/s]

119359 error


 97%|█████████████████████████████████████████████████████████████████████████▍  | 21499/22238 [24:20<00:38, 19.38it/s]

123184 error


100%|████████████████████████████████████████████████████████████████████████████| 22238/22238 [25:31<00:00, 14.52it/s]


In [10]:
big_df = pd.read_csv("f_with_context1.tsv", sep='\t')

In [14]:
big_df.drop_duplicates(subset=['4', 'в ходе', 'в + Sloc', 'способ', 'периферия',
       'ходе', 'S'], inplace=True)

In [11]:
big_df.head(10)

,59004,4,в ходе,в + Sloc,способ,периферия,избрать,ходе,S,loc,m,inan,sg,ход,ROOT,move,abstr,в,работ сантехник случайно пробил сквозную дыру примерно 50 × 50 см в стене между кухней и ванной .
0,138824,3,дыру,Sacc,результат,объект,"пробивать,пробить",дыру,S,acc,f,inan,sg,дыра,dobj,space,concr,в ходе работ сантехник случайно пробил сквозную,примерно 50 × 50 см в стене между кухней и ван...
1,138784,3,дыру,Sacc,результат,объект,"пробивать,пробить",дыру,S,acc,f,inan,sg,дыра,dobj,space,concr,в ходе работ сантехник случайно пробил сквозную,примерно 50 × 50 см в стене между кухней и ван...
2,138842,3,дыру,Sacc,результат,объект,"пробивать,пробить",дыру,S,acc,f,inan,sg,дыра,dobj,space,concr,в ходе работ сантехник случайно пробил сквозную,примерно 50 × 50 см в стене между кухней и ван...
3,105636,3,в дыру,в + Sacc,конечная точка,периферия,угодить,дыру,S,acc,f,inan,sg,дыра,dobj,space,concr,в ходе работ сантехник случайно пробил сквозную,примерно 50 × 50 см в стене между кухней и ван...
4,31995,3,на стене,на + Sloc,место,периферия,висеть,стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
5,32026,3,на стене,на + Sloc,место,периферия,висеть,стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
6,32040,3,на стене,на + Sloc,место,периферия,висеть,стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
7,108941,1,стене,Sloc,тема,контролер согласования,"достигать,достигнуть",стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
8,80761,3,стене,Sloc,пациенс,контролер согласования,"забрасывать,забросить",стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
9,34479,3,на стене,на + Sloc,место,периферия,"появиться,появляться",стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .


In [16]:
big_df = (big_df.T.reset_index().T.reset_index(drop=True)
            .set_axis(['ExIndex', 'PlaceC', 'Phrase', 'Form', 'Role', 'Rank',\
                       'KeyLexemes', 'Last_word', 'pos', 'case', 'gender', \
                       'animacy', 'number', 'lemma', 'link_name', 't', 'r', \
                       'left_context', 'right_context'], axis=1))  

In [17]:
big_df.columns

Index(['ExIndex', 'PlaceC', 'Phrase', 'Form', 'Role', 'Rank', 'KeyLexemes',
       'Last_word', 'pos', 'case', 'gender', 'animacy', 'number', 'lemma',
       'link_name', 't', 'r', 'left_context', 'right_context'],
      dtype='object')

In [19]:
big_df.head(10)

,ExIndex,PlaceC,Phrase,Form,Role,Rank,KeyLexemes,Last_word,pos,case,gender,animacy,number,lemma,link_name,t,r,left_context,right_context
0,59004,4,в ходе,в + Sloc,способ,периферия,избрать,ходе,S,loc,m,inan,sg,ход,ROOT,move,abstr,в,работ сантехник случайно пробил сквозную дыру ...
1,138824,3,дыру,Sacc,результат,объект,"пробивать,пробить",дыру,S,acc,f,inan,sg,дыра,dobj,space,concr,в ходе работ сантехник случайно пробил сквозную,примерно 50 × 50 см в стене между кухней и ван...
2,105636,3,в дыру,в + Sacc,конечная точка,периферия,угодить,дыру,S,acc,f,inan,sg,дыра,dobj,space,concr,в ходе работ сантехник случайно пробил сквозную,примерно 50 × 50 см в стене между кухней и ван...
3,31995,3,на стене,на + Sloc,место,периферия,висеть,стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
4,108941,1,стене,Sloc,тема,контролер согласования,"достигать,достигнуть",стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
5,80761,3,стене,Sloc,пациенс,контролер согласования,"забрасывать,забросить",стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
6,105862,4,на стене,на + Sloc,место,периферия,"укрепить,укреплять",стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
7,31974,1,на стене,на + Sloc,место,периферия,висеть,стене,S,loc,f,inan,sg,стена,nmod,NaN,concr,в ходе работ сантехник случайно пробил сквозну...,между кухней и ванной .
8,40241,3,в ванной,в + Sloc,место,периферия,"исчезать,исчезнуть",ванной,S,ins,f,inan,sg,ванная,conj,room,concr,в ходе работ сантехник случайно пробил сквозну...,.
9,42534,1,в ванной,в + Sloc,место,периферия,"бриться,побриться",ванной,S,ins,f,inan,sg,ванная,conj,room,concr,в ходе работ сантехник случайно пробил сквозну...,.


In [20]:
big_df.drop_duplicates(subset=['Phrase', 'Form', 'Role', 'Rank'], inplace=True)

In [21]:
big_df.info() # 8846084 -> 16965  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16965 entries, 0 to 17941
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ExIndex        16965 non-null  object
 1   PlaceC         16965 non-null  object
 2   Phrase         16965 non-null  object
 3   Form           16965 non-null  object
 4   Role           16965 non-null  object
 5   Rank           16965 non-null  object
 6   KeyLexemes     16965 non-null  object
 7   Last_word      16965 non-null  object
 8   pos            16965 non-null  object
 9   case           16965 non-null  object
 10  gender         16499 non-null  object
 11  animacy        16499 non-null  object
 12  number         16499 non-null  object
 13  lemma          16965 non-null  object
 14  link_name      16965 non-null  object
 15  t              10981 non-null  object
 16  r              16540 non-null  object
 17  left_context   16740 non-null  object
 18  right_context  16917 non-n

объединение ролей

In [39]:
pd.DataFrame(big_df['Role'].value_counts()).head(50)

,Role
пациенс,7813
место,3114
агенс,1194
инструмент,859
причина,807
признак,804
время,703
экспериенцер,426
адресат,238
цель,218


In [23]:
big_df.loc[(big_df['Role'] == 'субъект восприятия') | \
           (big_df['Role'] == 'субъект физиологической реакции') | \
           (big_df['Role'] == 'субъект физиологического ощущения') | \
           (big_df['Role'] == 'субъект ментального состояния') | \
           (big_df['Role'] == 'субъект психологического состояния'), 'Role'] = 'экспериенцер'

In [24]:
big_df.loc[(big_df['Role'] == 'тема') | \
           (big_df['Role'] == 'результат') | \
           (big_df['Role'] == 'пациенс перемещения') | \
           (big_df['Role'] == 'пациенс социального взаимодействия') | \
           (big_df['Role'] == 'пациенс социального отношения') | \
           (big_df['Role'] == 'предел') | \
           (big_df['Role'] == 'потенциальный результат') | \
           (big_df['Role'] == 'потенциальный пациенс') | \
           (big_df['Role'] == 'потенциальная ситуация') | \
           (big_df['Role'] == 'потенциальная угроза') | \
           (big_df['Role'] == 'подвергающаяся воздействию часть пациенса') | \
           (big_df['Role'] == 'часть субъекта физиологического ощущения') | \
           (big_df['Role'] == 'часть субъекта физиологической реакции') | \
           (big_df['Role'] == 'запах') | \
           (big_df['Role'] == 'стимул') | \
           
           (big_df['Role'] == 'часть темы в фокусе внимания') | \
           (big_df['Role'] == 'предмет высказывания') | \
           (big_df['Role'] == 'предмет мысли') | \
           (big_df['Role'] == 'содержание высказывания') | \
           (big_df['Role'] == 'содержание мысли') | \
           (big_df['Role'] == 'содержание действия') | \
           (big_df['Role'] == 'пациенс?'), 'Role'] = 'пациенс'

In [25]:
big_df.loc[(big_df['Role'] == 'расстояние') | \
           (big_df['Role'] == 'скорость') | \
           (big_df['Role'] == 'цена') | \
           (big_df['Role'] == 'вес') | \
           (big_df['Role'] == 'значение параметра'), 'Role'] = 'параметр'

In [26]:
big_df.loc[(big_df['Role'] == 'мотивировка') | \
           (big_df['Role'] == 'свойство причины'), 'Role'] = 'причина'

In [27]:
big_df.loc[(big_df['Role'] == 'статус') | \
           (big_df['Role'] == 'эталон') | \
           (big_df['Role'] == 'признак действия') | \
           (big_df['Role'] == 'образ действия') | \
           (big_df['Role'] == 'звук') | \
           (big_df['Role'] == 'степень') | \
           (big_df['Role'] == 'сила') | \
           (big_df['Role'] == 'аспект') | \
           (big_df['Role'] == 'ориентация'), 'Role'] = 'признак'

In [28]:
big_df.loc[(big_df['Role'] == 'время наступления') | \
           (big_df['Role'] == 'момент наступления события') | \
           (big_df['Role'] == 'узуальность') | \
           
           (big_df['Role'] == 'срок') | \
           
           (big_df['Role'] == 'длительность') | \
           (big_df['Role'] == 'момент времени') | \
           (big_df['Role'] == 'интервал') | \
           (big_df['Role'] == 'период времени') | \
           (big_df['Role'] == 'возраст') | \
           (big_df['Role'] == 'фаза'), 'Role'] = 'время'

In [37]:
big_df.loc[(big_df['Role'] == 'начальная точка') | \
           (big_df['Role'] == 'конечная точка') | \
           (big_df['Role'] == 'уровень') | \
           (big_df['Role'] == 'траектория') | \
           (big_df['Role'] == 'множество') | \
           (big_df['Role'] == 'направление') | \
           (big_df['Role'] == 'среда') | \
           (big_df['Role'] == 'социальная среда') | \
           (big_df['Role'] == 'сфера') | \
           (big_df['Role'] == 'опора') | \
           (big_df['Role'] == 'конечное состояние') | \
           (big_df['Role'] == 'точка контакта') | \
           (big_df['Role'] == 'вместилище'), 'Role'] = 'место'

In [30]:
big_df.loc[(big_df['Role'] == 'говорящий') | \
           (big_df['Role'] == 'каузатор') | \
           (big_df['Role'] == 'контрагент') | \
           (big_df['Role'] == 'посессор') | \
           (big_df['Role'] == 'конечный посессор') | \
           (big_df['Role'] == 'начальный посессор') | \
           (big_df['Role'] == 'исходный посессор') | \
           (big_df['Role'] == 'потенциальный эффектор') | \
           (big_df['Role'] == 'потенциальный контрагент') | \
           (big_df['Role'] == 'потенциальный агенс') | \
           (big_df['Role'] == 'контрагент социального отношения') | \
           (big_df['Role'] == 'источник') | \
           (big_df['Role'] == 'эффектор') | \
           (big_df['Role'] == 'часть агенса') | \
           (big_df['Role'] == 'субъект поведения') | \
           (big_df['Role'] == 'субъект социального отношения') | \
           (big_df['Role'] == 'субъект перемещения'), 'Role'] = 'агенс'

In [31]:
big_df.loc[(big_df['Role'] == 'часть субъекта восприятия') | \
           (big_df['Role'] == 'часть субъекта психологического состояния') | \
           (big_df['Role'] == 'часть субъекта ментального состояния') | \
           (big_df['Role'] == 'часть контрагента в фокусе внимания') | \
           (big_df['Role'] == 'квазиинструмент') | \
           (big_df['Role'] == 'средство') | \
           (big_df['Role'] == 'способ'), 'Role'] = 'инструмент'

In [32]:
big_df.loc[(big_df['Role'] == 'бенефициант') | \
           (big_df['Role'] == 'малефициант') | \
           (big_df['Role'] == 'часть адресата') | \
           (big_df['Role'] == 'аудитория'), 'Role'] = 'адресат'

In [33]:
big_df.loc[(big_df['Role'] == 'сопутствующая ситуация'), 'Role'] = 'фоновая ситуация'

In [34]:
big_df.loc[(big_df['Role'] == 'интенсивность') | (big_df['Role'] == 'точность') | (big_df['Role'] == 'охват'), 'Role'] = 'степень'

In [ ]:
big_df.to_csv('big_df2_most_recent_features.tsv', index=False, sep='\t')  